<a href="https://colab.research.google.com/github/Raesyitha27/FuzzyLogicPenjadwalanDosen/blob/main/FuzzyPenjadwalan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Variabel Input
preferensi_waktu = ctrl.Antecedent(np.arange(0, 11, 1), 'preferensi_waktu')  # Skala 0-10
ketersediaan_ruang = ctrl.Antecedent(np.arange(0, 11, 1), 'ketersediaan_ruang')  # Skala 0-10
jumlah_kelas = ctrl.Antecedent(np.arange(0, 11, 1), 'jumlah_kelas')  # Skala 0-10

# Variabel Output
optimalitas_jadwal = ctrl.Consequent(np.arange(0, 11, 1), 'optimalitas_jadwal')  # Skala 0-10

# Definisi Membership Function
preferensi_waktu['tidak_sesuai'] = fuzz.trapmf(preferensi_waktu.universe, [0, 0, 2, 4])
preferensi_waktu['cukup_sesuai'] = fuzz.trimf(preferensi_waktu.universe, [3, 5, 7])
preferensi_waktu['sangat_sesuai'] = fuzz.trapmf(preferensi_waktu.universe, [6, 8, 10, 10])

ketersediaan_ruang['tidak_tersedia'] = fuzz.trapmf(ketersediaan_ruang.universe, [0, 0, 2, 4])
ketersediaan_ruang['terbatas'] = fuzz.trimf(ketersediaan_ruang.universe, [3, 5, 7])
ketersediaan_ruang['tersedia'] = fuzz.trapmf(ketersediaan_ruang.universe, [6, 8, 10, 10])

jumlah_kelas['sedikit'] = fuzz.trapmf(jumlah_kelas.universe, [0, 0, 2, 4])
jumlah_kelas['sedang'] = fuzz.trimf(jumlah_kelas.universe, [3, 5, 7])
jumlah_kelas['banyak'] = fuzz.trapmf(jumlah_kelas.universe, [6, 8, 10, 10])

optimalitas_jadwal['kurang_optimal'] = fuzz.trapmf(optimalitas_jadwal.universe, [0, 0, 2, 4])
optimalitas_jadwal['cukup_fleksibel'] = fuzz.trimf(optimalitas_jadwal.universe, [3, 5, 7])
optimalitas_jadwal['optimal'] = fuzz.trapmf(optimalitas_jadwal.universe, [6, 8, 10, 10])

# Aturan Fuzzy
rule1 = ctrl.Rule(preferensi_waktu['sangat_sesuai'] & ketersediaan_ruang['tersedia'] & jumlah_kelas['sedikit'],
                  optimalitas_jadwal['optimal'])
rule2 = ctrl.Rule(preferensi_waktu['cukup_sesuai'] & ketersediaan_ruang['terbatas'] & jumlah_kelas['sedang'],
                  optimalitas_jadwal['cukup_fleksibel'])
rule3 = ctrl.Rule(preferensi_waktu['tidak_sesuai'] | ketersediaan_ruang['tidak_tersedia'],
                  optimalitas_jadwal['kurang_optimal'])
rule4 = ctrl.Rule(jumlah_kelas['banyak'] & ketersediaan_ruang['terbatas'],
                  optimalitas_jadwal['cukup_fleksibel'])

# Sistem Kontrol
jadwal_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
jadwal_simulasi = ctrl.ControlSystemSimulation(jadwal_ctrl)

# Input dari Pengguna
try:
    preferensi_input = float(input("Masukkan preferensi waktu dosen (0-10): "))
    ruang_input = float(input("Masukkan ketersediaan ruang kelas (0-10): "))
    kelas_input = float(input("Masukkan jumlah kelas yang diajar (0-10): "))

    if not (0 <= preferensi_input <= 10):
        raise ValueError("Nilai preferensi waktu harus dalam rentang 0-10.")
    if not (0 <= ruang_input <= 10):
        raise ValueError("Nilai ketersediaan ruang harus dalam rentang 0-10.")
    if not (0 <= kelas_input <= 10):
        raise ValueError("Nilai jumlah kelas harus dalam rentang 0-10.")

    # Masukkan input ke simulasi
    jadwal_simulasi.input['preferensi_waktu'] = preferensi_input
    jadwal_simulasi.input['ketersediaan_ruang'] = ruang_input
    jadwal_simulasi.input['jumlah_kelas'] = kelas_input

    # Hitung output
    jadwal_simulasi.compute()

    # Output hasil
    hasil_optimalitas = jadwal_simulasi.output['optimalitas_jadwal']
    if hasil_optimalitas <= 4:
        deskripsi_optimalitas = "Kurang Optimal"
    elif 4 < hasil_optimalitas <= 7:
        deskripsi_optimalitas = "Cukup Fleksibel"
    else:
        deskripsi_optimalitas = "Optimal"

    print(f"\nPrediksi Optimalitas Jadwal (0=kurang optimal, 10=optimal): {hasil_optimalitas:.2f}")
    print(f"Kondisi Jadwal: {deskripsi_optimalitas}")

except ValueError as e:
    print(f"Input tidak valid: {e}")


In [3]:
! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fungsi Keanggotaan untuk Preferensi Waktu
preferensi_waktu = np.arange(0, 11, 1)
tidak_sesuai = np.piecewise(preferensi_waktu,
                            [preferensi_waktu <= 2,
                             (preferensi_waktu > 2) & (preferensi_waktu <= 4),
                             preferensi_waktu > 4],
                            [1, lambda x: (4 - x) / (4 - 2), 0])
cukup_sesuai = np.piecewise(preferensi_waktu,
                            [(preferensi_waktu >= 3) & (preferensi_waktu <= 5),
                             (preferensi_waktu > 5) & (preferensi_waktu <= 7)],
                            [lambda x: (x - 3) / (5 - 3), lambda x: (7 - x) / (7 - 5)])
sangat_sesuai = np.piecewise(preferensi_waktu,
                             [(preferensi_waktu >= 6) & (preferensi_waktu <= 8),
                              preferensi_waktu > 8],
                             [lambda x: (x - 6) / (8 - 6), 1])

# Fungsi Keanggotaan untuk Ketersediaan Ruang
ketersediaan_ruang = np.arange(0, 11, 1)
tidak_tersedia = np.piecewise(ketersediaan_ruang,
                              [ketersediaan_ruang <= 3,
                               (ketersediaan_ruang > 3) & (ketersediaan_ruang <= 5),
                               ketersediaan_ruang > 5],
                              [1, lambda x: (5 - x) / (5 - 3), 0])
terbatas = np.piecewise(ketersediaan_ruang,
                        [(ketersediaan_ruang >= 4) & (ketersediaan_ruang <= 6),
                         (ketersediaan_ruang > 6) & (ketersediaan_ruang <= 8)],
                        [lambda x: (x - 4) / (6 - 4), lambda x: (8 - x) / (8 - 6)])
tersedia = np.piecewise(ketersediaan_ruang,
                        [(ketersediaan_ruang >= 7) & (ketersediaan_ruang <= 9),
                         ketersediaan_ruang > 9],
                        [lambda x: (x - 7) / (9 - 7), 1])

# Fungsi Keanggotaan untuk Jumlah Kelas
jumlah_kelas = np.arange(0, 11, 1)
sedikit = np.piecewise(jumlah_kelas,
                       [jumlah_kelas <= 3,
                        (jumlah_kelas > 3) & (jumlah_kelas <= 5),
                        jumlah_kelas > 5],
                       [1, lambda x: (5 - x) / (5 - 3), 0])
sedang = np.piecewise(jumlah_kelas,
                      [(jumlah_kelas >= 4) & (jumlah_kelas <= 6),
                       (jumlah_kelas > 6) & (jumlah_kelas <= 8)],
                      [lambda x: (x - 4) / (6 - 4), lambda x: (8 - x) / (8 - 6)])
banyak = np.piecewise(jumlah_kelas,
                      [(jumlah_kelas >= 7) & (jumlah_kelas <= 9),
                       jumlah_kelas > 9],
                      [lambda x: (x - 7) / (9 - 7), 1])

# Fungsi Keanggotaan untuk Optimalitas Jadwal
optimalitas_jadwal = np.arange(0, 11, 1)
kurang_optimal = np.piecewise(optimalitas_jadwal,
                              [optimalitas_jadwal <= 3,
                               (optimalitas_jadwal > 3) & (optimalitas_jadwal <= 5),
                               optimalitas_jadwal > 5],
                              [1, lambda x: (5 - x) / (5 - 3), 0])
cukup_fleksibel = np.piecewise(optimalitas_jadwal,
                               [(optimalitas_jadwal >= 4) & (optimalitas_jadwal <= 6),
                                (optimalitas_jadwal > 6) & (optimalitas_jadwal <= 8)],
                               [lambda x: (x - 4) / (6 - 4), lambda x: (8 - x) / (8 - 6)])
optimal = np.piecewise(optimalitas_jadwal,
                       [(optimalitas_jadwal >= 7) & (optimalitas_jadwal <= 9),
                        optimalitas_jadwal > 9],
                       [lambda x: (x - 7) / (9 - 7), 1])

# Plot Semua Fungsi Keanggotaan
variables = {
    "Preferensi Waktu": (preferensi_waktu, [tidak_sesuai, cukup_sesuai, sangat_sesuai], ["Tidak Sesuai", "Cukup Sesuai", "Sangat Sesuai"]),
    "Ketersediaan Ruang": (ketersediaan_ruang, [tidak_tersedia, terbatas, tersedia], ["Tidak Tersedia", "Terbatas", "Tersedia"]),
    "Jumlah Kelas": (jumlah_kelas, [sedikit, sedang, banyak], ["Sedikit", "Sedang", "Banyak"]),
    "Optimalitas Jadwal": (optimalitas_jadwal, [kurang_optimal, cukup_fleksibel, optimal], ["Kurang Optimal", "Cukup Fleksibel", "Optimal"])
}

for name, (x, memberships, labels) in variables.items():
    plt.figure(figsize=(8, 5))
    for membership, label in zip(memberships, labels):
        plt.plot(x, membership, label=label)
    plt.title(f"Fungsi Keanggotaan {name}")
    plt.xlabel("Nilai")
    plt.ylabel("Derajat Keanggotaan")
    plt.legend()
    plt.grid()
    plt.show()